In [12]:
import numpy as np
import random, math
import copy

Dimension = 4
qubits = 2
A = np.array([[-9, -2,-2,9], [-3, 10,-3,-3], [-8,6,-9,-8], [-8,8,10,9]]) # Row vectors
b = np.array([-29, 19, 30, -19]) # Column vectors

# 16 qubits with integer solutions
# x = q1 + 2q2 - q3 - 2q4
# x = {{0}, {1}, {0}, {-3}}
# mininum -2463

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
                
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q|
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[  -14.   872.     0.     0.  -248.  -496.   248.   496.    38.    76.
    -38.   -76.  -160.  -320.   160.   320.]
 [    0.   408.     0.     0.  -496.  -992.   496.   992.    76.   152.
    -76.  -152.  -320.  -640.   320.   640.]
 [    0.     0.   450.   872.   248.   496.  -248.  -496.   -38.   -76.
     38.    76.   160.   320.  -160.  -320.]
 [    0.     0.     0.  1336.   496.   992.  -496.  -992.   -76.  -152.
     76.   152.   320.   640.  -320.  -640.]
 [    0.     0.     0.     0.  -348.   816.     0.     0.     0.     0.
      0.     0.   -48.   -96.    48.    96.]
 [    0.     0.     0.     0.     0.  -288.     0.     0.     0.     0.
      0.     0.   -96.  -192.    96.   192.]
 [    0.     0.     0.     0.     0.     0.   756.   816.     0.     0.
      0.     0.    48.    96.   -48.   -96.]
 [    0.     0.     0.     0.     0.     0.     0.  1920.     0.     0.
      0.     0.    96.   192.   -96.  -192.]
 [    0.     0.     0.     0.     0.     0.     0.

In [2]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-14.0, ('q2','q2'):408.0, ('q3','q3'):450.0, ('q4','q4'):1336.0, ('q5','q5'):-348.0, ('q6','q6'):-288.0, ('q7','q7'):756.0, ('q8','q8'):1920.0, ('q9','q9'):1112.0, ('q10','q10'):2612.0, ('q11','q11'):-724.0, ('q12','q12'):-1060.0, ('q13','q13'):1693.0, ('q14','q14'):3856.0, ('q15','q15'):-1223.0, ('q16','q16'):-1976.0}

quadratic = {('q1','q2'):872.0, ('q1','q5'):-248.0, ('q1','q6'):-496.0, ('q1','q7'):248.0, ('q1','q8'):496.0, ('q1','q9'):38.0, ('q1','q10'):76.0, ('q1','q11'):-38.0, ('q1','q12'):-76.0, ('q1','q13'):-160.0, ('q1','q14'):-320.0, ('q1','q15'):160.0, ('q1','q16'):320.0, ('q2','q5'):-496.0, ('q2','q6'):-992.0, ('q2','q7'):496.0, ('q2','q8'):992.0, ('q2','q9'):76.0, ('q2','q10'):152.0, ('q2','q11'):-76.0, ('q2','q12'):-152.0, ('q2','q13'):-320.0, ('q2','q14'):-640.0, ('q2','q15'):320.0, ('q2','q16'):640.0, ('q3','q4'):872.0, ('q3','q5'):248.0, ('q3','q6'):496.0, ('q3','q7'):-248.0, ('q3','q8'):-496.0, ('q3','q9'):-38.0, ('q3','q10'):-76.0, ('q3','q11'):38.0, ('q3','q12'):76.0, ('q3','q13'):160.0, ('q3','q14'):320.0, ('q3','q15'):-160.0, ('q3','q16'):-320.0, ('q4','q5'):496.0, ('q4','q6'):992.0, ('q4','q7'):-496.0, ('q4','q8'):-992.0, ('q4','q9'):-76.0, ('q4','q10'):-152.0, ('q4','q11'):76.0, ('q4','q12'):152.0, ('q4','q13'):320.0, ('q4','q14'):640.0, ('q4','q15'):-320.0, ('q4','q16'):-640.0, ('q5','q6'):816.0, ('q5','q13'):-48.0, ('q5','q14'):-96.0, ('q5','q15'):48.0, ('q5','q16'):96.0, ('q6','q13'):-96.0, ('q6','q14'):-192.0, ('q6','q15'):96.0, ('q6','q16'):192.0, ('q7','q8'):816.0, ('q7','q13'):48.0, ('q7','q14'):96.0, ('q7','q15'):-48.0, ('q7','q16'):-96.0, ('q8','q13'):96.0, ('q8','q14'):192.0, ('q8','q15'):-96.0, ('q8','q16'):-192.0, ('q9','q10'):776.0, ('q9','q13'):306.0, ('q9','q14'):612.0, ('q9','q15'):-306.0, ('q9','q16'):-612.0, ('q10','q13'):612.0, ('q10','q14'):1224.0, ('q10','q15'):-612.0, ('q10','q16'):-1224.0, ('q11','q12'):776.0, ('q11','q13'):-306.0, ('q11','q14'):-612.0, ('q11','q15'):306.0, ('q11','q16'):612.0, ('q12','q13'):-612.0, ('q12','q14'):-1224.0, ('q12','q15'):612.0, ('q12','q16'):1224.0, ('q13','q14'):940.0, ('q15','q16'):940.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
0    0   0   0   0   0   0   1   1  0  0  0  1  0  0  0  0 -2463.0      72 ...
1    1   0   1   0   0   0   0   1  0  0  0  0  1  0  0  0 -2412.0      26 ...
119  1   0   1   0   0   0   0   1  0  0  0  0  1  0  0  0 -2412.0       1 ...
2    0   0   1   0   0   0   0   1  0  0  0  1  0  0  0  0 -2340.0      51 ...
3    1   0   1   0   0   0   0   1  0  0  0  1  0  0  0  0 -2320.0      16 ...
4    1   0   0   1   0   0   1   0  0  0  0  0  1  0  0  0 -2289.0      10 ...
5    1   0   0   0   0   0   1   1  0  0  0  0  1  0  0  0 -2289.0       6 ...
6    0   0   0   0   0   0   1   1  0  1  0  0  0  0  0  0 -2289.0      23 ...
7    0   0   0   1   0   0   1   0  1  0  0  0  1  0  0  0 -2279.0      21 ...
8    0   0   0   0   0   0   1   1  0  0  0  1  0  0  0  1 -2269.0     145 ...
9    0   0   1   0   0   0   1   1  0  0  0  1  0  0  0  0 -2269.0      14 ...
10   1   0   0   0   0   0   0   1  0  0  0  0  1  0